In [25]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import json

import pickle

from tssplit import tssplit
import re

In [26]:
def remove_duplicates(l):
    return list(set(l))


In [27]:
df = pd.read_pickle("p:/uni/nlp/WashingtonPost_v4/data/v5/12_Sorted.pkl")

In [28]:
pattern_auth = "—\s*\w*"
pattern_at_empty = "" #(\:*\s*'')
drop_list = []
index = 0
i = 0



for auth in df["author"]:
    at = re.search(pattern_auth, str(auth))
    #at2 = re.search(pattern_at_empty, str(auth))
    if at or len(str(auth)) <= 4:
      drop_list.append(i)
      #print(i)
    if auth == "":
      drop_list.append(i)
    i+= 1

In [29]:
df = df.drop(remove_duplicates(drop_list))

In [30]:

df = df.sort_index()
df = df.reset_index(drop=True)


In [31]:
pattern_paragraph = "\'subtype\': \'paragraph\'"
pattern_date = "\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z"
temp_List_meta = []
temp_List_meta2 = []

temp_List_meta4 = []
temp_List_content = []
temp_List_content2 = []

meta_List = []
content_List = []

j= 0

for article in df["contents"]:
    temp_List_meta = []
    temp_List_content = []
    temp_List_content2 = []
    temp_List_meta2 = []
    temp_List_meta4 = []
    
    for eintrag in article:
        x = re.search(pattern_paragraph, str(eintrag))
        if x:
            temp_List_content = [eintrag]
            for t in temp_List_content:
                val = str(t)
                val3 = tssplit(val,quote="'", delimiter=",")
                t4 = val3[0][10:]
                temp_List_content2.append(t4)
        else:
            temp_List_meta = [eintrag]
            for t in temp_List_meta:
            
                val2 = str(t)
                val6 = val2.split(",")
                t8 = val6[0][13:]
                temp_List_meta2.append(t8)
    try:
        temp_List_meta4.append(temp_List_meta2[0])
        temp_List_meta4.append(temp_List_meta2[1])
    except:
        continue 
    
    temp_List_meta4.append(df["author"][j])
                   
    for m in temp_List_meta2:                
        d = re.search(pattern_date, str(m))          
        if d:
            temp_List_meta4.append(m)
        else:
            continue
    
    if len(temp_List_meta4) >= 3 and len(temp_List_meta4) <= 4 and len(temp_List_content2) > 3:
        #if temp_List_meta4[0]
        meta_List.append(temp_List_meta4)    
        content_List.append("".join(temp_List_content2))
    
    j+=1        


In [32]:
dfMeta = pd.DataFrame(meta_List)
dfContent = pd.DataFrame(content_List)

In [33]:
dfContent = dfContent.set_axis(["content"], axis=1, inplace = False)

dfMeta = dfMeta.set_axis(["Kategorie","Titel","Autor","Datum"], axis=1, inplace = False)

In [34]:
dfContent["id"] = dfContent.index
dfMeta["id1"] = dfMeta.index

In [35]:
dfData = pd.merge(dfMeta,dfContent, left_on="id1", right_on="id", how="left").drop("id1",axis=1)

In [36]:
dfData.to_pickle("p:/uni/nlp/WashingtonPost_v4/data/v5/RAWData.pkl")

TypeError: expected string or bytes-like object